In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

In [3]:
df = pd.read_csv(r'C:\Users\Merrick\Downloads\Resampled_data.csv', error_bad_lines=False)

In [9]:
df.isnull().any

<bound method DataFrame.any of         time    bid    ask  News_count
0      False  False  False       False
1      False  False  False       False
2      False  False  False       False
3      False  False  False       False
4      False  False  False       False
5      False  False  False       False
6      False  False  False       False
7      False  False  False       False
8      False  False  False       False
9      False  False  False       False
10     False  False  False       False
11     False  False  False       False
12     False  False  False       False
13     False  False  False       False
14     False  False  False       False
15     False  False  False       False
16     False  False  False       False
17     False  False  False       False
18     False  False  False       False
19     False  False  False       False
20     False  False  False       False
21     False  False  False       False
22     False  False  False       False
23     False  False  False       

In [ ]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    # SnowballStemmer, PorterStemmer, LemmanizerStemmer
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
return text
df['titles'] = df['titles'].map(lambda x: clean_text(x))



In [ ]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['titles'])
sequences = tokenizer.texts_to_sequences(df['titles'])
data = pad_sequences(sequences, maxlen=50)

In [ ]:
# model = Sequential()
# model.add(Embedding(20000, 100, input_length=50))
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(data, np.array(labels), validation_split=0.4, epochs=3)

In [ ]:
# word_embds = model.layers[0].get_weights()[
# ist = []
# for word, i in tokenizer.word_index.items():
#     word_list.append(word)
# X_embedded = TSNE(n_components=2).fit_transform(word_weights)
# number_of_words = 1000
# trace = go.Scatter(
#     x = X_embedded[0:number_of_words,0], 
#     y = X_embedded[0:number_of_words, 1],
#     mode = 'markers',
#     text= word_list[0:number_of_words]
# )
# layout = dict(title= 't-SNE 1 vs t-SNE 2 for sirst 1000 words ',
#               yaxis = dict(title='t-SNE 2'),
#               xaxis = dict(title='t-SNE 1'),
#               hovermode= 'closest')
# fig = dict(data = [trace], layout= layout)
# py.iplot(fig)

In [ ]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

In [ ]:
## Get weights
conv_embds = model_conv.layers[0].get_weights()[0]
## Plotting function
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)

## Visualize words in two dimensions 
glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)
plot_words(glove_tsne_embds, 0, 2000, 1)